In [1]:
import sys
import tensorflow as tf
sys.path.append('../../generative-adversarial-nets/')
from ganetwork import GAN, CGAN
from sklearn import datasets
from sklearn.preprocessing import minmax_scale, LabelBinarizer

In [2]:
ds = datasets.load_iris()
X, y = minmax_scale(ds.data), LabelBinarizer().fit_transform(ds.target)

In [3]:
discriminator_layers=[(4, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(6, None), (5, tf.nn.tanh), (4, None)]
gan = GAN(discriminator_layers, generator_layers)
gan.train(X, nb_epoch=50, batch_size=5, discriminator_steps=3)

Epoch: 0, discriminator loss: 1.5570275783538818, generator loss: 0.36414971947669983
Epoch: 1, discriminator loss: 1.4365156888961792, generator loss: 0.486114501953125
Epoch: 2, discriminator loss: 1.3996174335479736, generator loss: 0.5563958883285522
Epoch: 3, discriminator loss: 1.386549711227417, generator loss: 0.6266053915023804
Epoch: 4, discriminator loss: 1.412413477897644, generator loss: 0.639626145362854
Epoch: 5, discriminator loss: 1.4342918395996094, generator loss: 0.6600140333175659
Epoch: 6, discriminator loss: 1.432455062866211, generator loss: 0.6395387649536133
Epoch: 7, discriminator loss: 1.4026256799697876, generator loss: 0.6956972479820251
Epoch: 8, discriminator loss: 1.4157814979553223, generator loss: 0.6634725332260132
Epoch: 9, discriminator loss: 1.4599792957305908, generator loss: 0.7059472799301147
Epoch: 10, discriminator loss: 1.4524977207183838, generator loss: 0.6635796427726746
Epoch: 11, discriminator loss: 1.444462776184082, generator loss: 0.

In [4]:
discriminator_layers=[(7, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(9, None), (5, tf.nn.tanh), (4, None)]
cgan = CGAN(discriminator_layers, generator_layers)
cgan.train(X, y, nb_epoch=50, batch_size=5)

Epoch: 0, discriminator loss: 1.8388135433197021, generator loss: 0.22197838127613068
Epoch: 1, discriminator loss: 1.7347800731658936, generator loss: 0.26084405183792114
Epoch: 2, discriminator loss: 1.645878791809082, generator loss: 0.3028460741043091
Epoch: 3, discriminator loss: 1.5836520195007324, generator loss: 0.3411028981208801
Epoch: 4, discriminator loss: 1.5391974449157715, generator loss: 0.3736936151981354
Epoch: 5, discriminator loss: 1.5077012777328491, generator loss: 0.4034876823425293
Epoch: 6, discriminator loss: 1.4826488494873047, generator loss: 0.4292951226234436
Epoch: 7, discriminator loss: 1.4611555337905884, generator loss: 0.45460158586502075
Epoch: 8, discriminator loss: 1.4470641613006592, generator loss: 0.4766998887062073
Epoch: 9, discriminator loss: 1.4348974227905273, generator loss: 0.5005998611450195
Epoch: 10, discriminator loss: 1.4244184494018555, generator loss: 0.5186344981193542
Epoch: 11, discriminator loss: 1.4145386219024658, generator l